# SGR Quality: Executive Notebook

Цель: дать понятную аналитику качества продаж на уровне диалогов и кейсов.

Контракт (консистентен с `sgr_core.py` и `stability_case_review.md`):
- единица оценки: `conversation_id`;
- один bundled `evaluator` + один bundled `judge` на диалог;
- `greeting` считается только в первых 3 сообщениях продавца;
- `upsell` и `empathy` считаются по всему диалогу;
- полный audit trace сохраняется для каждого запуска.


In [1]:
from __future__ import annotations

import json
import sqlite3
from pathlib import Path

import pandas as pd
from IPython.display import display

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 240)


def resolve_db_path() -> Path:
    cwd = Path.cwd().resolve()
    candidates = [cwd / "dialogs.db", cwd.parent / "dialogs.db"]
    candidates.extend(parent / "dialogs.db" for parent in cwd.parents)
    for path in candidates:
        if path.exists():
            return path
    raise FileNotFoundError("dialogs.db not found. Run: make init-fresh && make scan")


def qdf(sql: str, params: tuple[object, ...] = ()) -> pd.DataFrame:
    return pd.read_sql_query(sql, conn, params=params)


DB_PATH = resolve_db_path()
conn = sqlite3.connect(DB_PATH)
conn.row_factory = sqlite3.Row

run_row = conn.execute(
    """
    SELECT run_id, summary_json
    FROM scan_runs
    WHERE status='success'
    ORDER BY started_at_utc DESC
    LIMIT 1
    """
).fetchone()
if run_row is None:
    raise ValueError("No successful run found. Run: make scan")

RUN_ID = str(run_row["run_id"])
SUMMARY = json.loads(str(run_row["summary_json"] or "{}"))

print(f"Используется последний успешный run_id: {RUN_ID}")
print(f"База: {DB_PATH}")
print(f"metrics_version: {SUMMARY.get('metrics_version')}")


Используется последний успешный run_id: scan_b400f5cf1e1e
База: /Users/ablackman/go/src/github.com/tetraminz/sales_protocol/dialogs.db
metrics_version: v5_dialog_level_bundle


## 0) Глоссарий

Ключевые термины, чтобы одинаково читать таблицы и выводы ниже.


In [2]:
glossary = pd.DataFrame(
    [
        {"Термин": "Диалог", "Что это значит": "Отдельная переписка с клиентом (`conversation_id`)."},
        {"Термин": "Правило", "Что это значит": "Проверка качества: Приветствие, Допродажа, Эмпатия."},
        {"Термин": "Результат", "Что это значит": "`Получилось` — правило выполнено; `Не получилось` — нет."},
        {"Термин": "Нужна проверка QA", "Что это значит": "`Да` — judge не согласился с evaluator (`judge_label=0`)."},
        {"Термин": "Причина", "Что это значит": "Короткая типовая формулировка, почему получился такой результат."},
        {"Термин": "Пояснение", "Что это значит": "Развернутое объяснение evaluator для конкретного кейса."},
        {"Термин": "Комментарий QA", "Что это значит": "Аргументация judge, если нужна дополнительная проверка."},
        {"Термин": "Цитата", "Что это значит": "Фраза из диалога, на которую опиралась оценка."},
        {"Термин": "Рекомендация", "Что это значит": "Что делать дальше, чтобы улучшить качество ответа."},
    ]
)

display(glossary)


,Термин,Что это значит
0,Диалог,Отдельная переписка с клиентом (`conversation_id`).
1,Правило,"Проверка качества: Приветствие, Допродажа, Эмпатия."
2,Результат,`Получилось` — правило выполнено; `Не получилось` — нет.
3,Нужна проверка QA,`Да` — judge не согласился с evaluator (`judge_label=0`).
4,Причина,"Короткая типовая формулировка, почему получился такой результат."
5,Пояснение,Развернутое объяснение evaluator для конкретного кейса.
6,Комментарий QA,"Аргументация judge, если нужна дополнительная проверка."
7,Цитата,"Фраза из диалога, на которую опиралась оценка."
8,Рекомендация,"Что делать дальше, чтобы улучшить качество ответа."


## 1) KPI latest run

Быстрая управленческая сводка по последнему успешному запуску.


In [3]:
llm_calls = qdf(
    """
    SELECT phase, COUNT(*) AS calls
    FROM llm_calls
    WHERE run_id=?
    GROUP BY phase
    ORDER BY phase
    """,
    (RUN_ID,),
)

phase_calls = {str(r["phase"]): int(r["calls"]) for _, r in llm_calls.iterrows()} if not llm_calls.empty else {}

greeting_violations_df = qdf(
    """
    SELECT COUNT(*) AS violations
    FROM scan_results
    WHERE run_id=?
      AND rule_key='greeting'
      AND eval_hit=1
      AND COALESCE(evidence_message_order, 999) > 3
    """,
    (RUN_ID,),
)
greeting_violations_n = int(greeting_violations_df.iloc[0]["violations"]) if not greeting_violations_df.empty else 0
greeting_window_kpi = (
    f"OK ({greeting_violations_n} нарушений)"
    if greeting_violations_n == 0
    else f"Требует внимания ({greeting_violations_n} нарушений)"
)

kpi = pd.DataFrame(
    [
        {"Показатель": "Run ID", "Значение": RUN_ID},
        {"Показатель": "Версия метрик", "Значение": SUMMARY.get("metrics_version", "n/a")},
        {"Показатель": "Выбрано диалогов", "Значение": int(SUMMARY.get("selected_conversations", 0))},
        {"Показатель": "Оценено диалогов", "Значение": int(SUMMARY.get("evaluated_conversations", 0))},
        {
            "Показатель": "Пропущено без сообщений продавца",
            "Значение": int(SUMMARY.get("skipped_conversations_without_seller", 0)),
        },
        {"Показатель": "Проверок правил записано", "Значение": int(SUMMARY.get("inserted", 0))},
        {"Показатель": "Покрытие QA", "Значение": f"{float(SUMMARY.get('judge_coverage', 0.0)):.1%}"},
        {"Показатель": "Ожидаемые LLM-вызовы", "Значение": int(SUMMARY.get("evaluated_conversations", 0)) * 2},
        {"Показатель": "LLM-вызовы evaluator", "Значение": int(phase_calls.get("evaluator", 0))},
        {"Показатель": "LLM-вызовы judge", "Значение": int(phase_calls.get("judge", 0))},
        {"Показатель": "Окно приветствия", "Значение": greeting_window_kpi},
    ]
)

display(kpi)


,Показатель,Значение
0,Run ID,scan_b400f5cf1e1e
1,Версия метрик,v5_dialog_level_bundle
2,Выбрано диалогов,5
3,Оценено диалогов,5
4,Пропущено без сообщений продавца,0
5,Проверок правил записано,15
6,Покрытие QA,100.0%
7,Ожидаемые LLM-вызовы,10
8,LLM-вызовы evaluator,5
9,LLM-вызовы judge,5


## 2) Rule Stats (по диалогам)

Компактная сводка по каждому правилу без технических служебных полей.


In [4]:
RULE_LABELS = {
    "greeting": "Приветствие",
    "upsell": "Допродажа",
    "empathy": "Эмпатия",
}

rule_metrics = qdf(
    """
    SELECT
      rule_key,
      eval_total,
      eval_true,
      ROUND(evaluator_hit_rate, 4) AS evaluator_hit_rate,
      ROUND(judge_correctness, 4) AS judge_correctness,
      ROUND(judge_coverage, 4) AS judge_coverage
    FROM scan_metrics
    WHERE run_id=?
    ORDER BY rule_key
    """,
    (RUN_ID,),
)

if rule_metrics.empty:
    print("Нет данных по правилам для выбранного run.")
else:
    rule_stats_view = pd.DataFrame(
        {
            "Правило": rule_metrics["rule_key"].map(lambda x: RULE_LABELS.get(str(x), "Неизвестное правило")),
            "Проверено": rule_metrics["eval_total"].astype(int),
            "Сработало": rule_metrics["eval_true"].astype(int),
            "Доля срабатываний": (rule_metrics["evaluator_hit_rate"] * 100).map(lambda x: f"{float(x):.1f}%"),
            "Согласие QA": (rule_metrics["judge_correctness"] * 100).map(lambda x: f"{float(x):.1f}%"),
            "Покрытие QA": (rule_metrics["judge_coverage"] * 100).map(lambda x: f"{float(x):.1f}%"),
        }
    )
    display(rule_stats_view)


,Правило,Проверено,Сработало,Доля срабатываний,Согласие QA,Покрытие QA
0,Эмпатия,5,5,100.0%,100.0%,100.0%
1,Приветствие,5,5,100.0%,100.0%,100.0%
2,Допродажа,5,5,100.0%,100.0%,100.0%


## 3) Кейсы: что получилось / что не получилось

Фокус на практических кейсах: понятный статус, причина, цитата и действие.


In [5]:
CASE_COLUMNS = [
    "Диалог",
    "Правило",
    "Результат",
    "Нужна проверка QA",
    "Причина",
    "Пояснение",
    "Комментарий QA",
    "Цитата",
    "Рекомендация",
]

REASON_LABELS = {
    "greeting_present": "Приветствие в нужном окне",
    "greeting_missing": "Приветствие не найдено",
    "greeting_late": "Приветствие слишком поздно",
    "upsell_offer": "Есть уместная допродажа",
    "upsell_missing": "Нет предложения следующего шага",
    "discount_without_upsell": "Есть скидка, но нет допродажи",
    "empathy_acknowledged": "Эмпатия выражена",
    "courtesy_without_empathy": "Вежливость без эмпатии",
    "informational_without_empathy": "Информирование без эмпатии",
}

RULE_RECOMMENDATIONS = {
    "greeting": "Добавить приветствие в первых трех сообщениях продавца.",
    "upsell": "Предложить конкретный следующий платный шаг по ситуации клиента.",
    "empathy": "Явно признать ситуацию клиента перед решением.",
}

cases = qdf(
    """
    SELECT
      sr.conversation_id,
      sr.rule_key,
      sr.eval_hit,
      sr.judge_label,
      sr.eval_reason_code,
      sr.eval_reason,
      sr.judge_rationale,
      sr.evidence_quote,
      COALESCE(anchor.text, '') AS evidence_message_text
    FROM scan_results sr
    LEFT JOIN messages anchor ON anchor.message_id = sr.evidence_message_id
    WHERE sr.run_id=?
    ORDER BY sr.rule_key, sr.conversation_id
    """,
    (RUN_ID,),
)


def normalize_reason(code: object, fallback: object) -> str:
    code_str = str(code or "").strip()
    if code_str:
        return REASON_LABELS.get(code_str, code_str)
    fallback_str = str(fallback or "").strip()
    return fallback_str or "Без уточнения"


def pick_quote(evidence_quote: object, evidence_text: object) -> str:
    quote = str(evidence_quote or "").strip()
    if quote:
        return quote
    text = str(evidence_text or "").strip()
    return text


def build_recommendation(rule_key: object, eval_hit: object) -> str:
    if int(eval_hit) == 1:
        return "Подход корректный"
    return RULE_RECOMMENDATIONS.get(str(rule_key), "Уточнить действие по правилу.")


if cases.empty:
    cases_export = pd.DataFrame(columns=CASE_COLUMNS)
else:
    prepared = cases.copy()
    prepared["Диалог"] = prepared["conversation_id"].astype(str)
    prepared["Правило"] = prepared["rule_key"].map(lambda x: RULE_LABELS.get(str(x), "Неизвестное правило"))
    prepared["Результат"] = prepared["eval_hit"].map(lambda x: "Получилось" if int(x) == 1 else "Не получилось")
    prepared["Нужна проверка QA"] = prepared["judge_label"].map(lambda x: "Да" if int(x) == 0 else "Нет")
    prepared["Причина"] = [
        normalize_reason(code, reason)
        for code, reason in zip(prepared["eval_reason_code"], prepared["eval_reason"])
    ]
    prepared["Пояснение"] = prepared["eval_reason"].fillna("").astype(str)
    prepared["Комментарий QA"] = prepared["judge_rationale"].fillna("").astype(str)
    prepared["Цитата"] = [
        pick_quote(eq, et)
        for eq, et in zip(prepared["evidence_quote"], prepared["evidence_message_text"])
    ]
    prepared["Рекомендация"] = [
        build_recommendation(rule_key, hit)
        for rule_key, hit in zip(prepared["rule_key"], prepared["eval_hit"])
    ]
    cases_export = prepared[CASE_COLUMNS].copy()

print(f"Всего кейсов в выбранном run: {len(cases_export)}")

overview = cases_export.head(20)
print("Обзор (первые 20 кейсов):")
display(overview)

failed_cases = cases_export[cases_export["Результат"] == "Не получилось"].head(20)
if failed_cases.empty:
    print("Неуспешных кейсов в выбранном run нет.")
else:
    print("Неуспешные кейсы (до 20):")
    display(failed_cases)

qa_cases = cases_export[cases_export["Нужна проверка QA"] == "Да"].head(20)
if qa_cases.empty:
    print("Кейсов с QA-расхождением в выбранном run нет.")
else:
    print("Кейсы с QA-расхождением (до 20):")
    display(qa_cases)


Всего кейсов в выбранном run: 15
Обзор (первые 20 кейсов):


,Диалог,Правило,Результат,Нужна проверка QA,Причина,Пояснение,Комментарий QA,Цитата,Рекомендация
0,modamart__0_transcript,Эмпатия,Получилось,Нет,Эмпатия выражена,Продавец явно признаёт и понимает опасения клиента о качестве и размере товара.,Есть явное признание и понимание беспокойств клиента.,I completely understand your concerns.,Подход корректный
1,modamart__1_transcript,Эмпатия,Получилось,Нет,Эмпатия выражена,Продавец признал опасения клиента по поводу теплоты и водостойкости курток.,"Продавец признал и ответил на обеспокоенность клиента по поводу свойств утеплителя, что является проявлением эмпатии.","That’s a great point. Down is incredibly warm, but it can lose its insulating properties when wet. Synthetic insulation, on the other hand, retains warmth even when damp.",Подход корректный
2,modamart__2_transcript,Эмпатия,Получилось,Нет,Эмпатия выражена,Продавец явно признал ситуацию клиента и его опасения.,Продавец признал ситуацию клиента словами 'I understand your concerns.'.,I understand your concerns.,Подход корректный
3,modamart__3_transcript,Эмпатия,Получилось,Нет,Эмпатия выражена,Продавец выразил сочувствие по поводу проблем с онлайн-шопингом клиента.,Продавец выразил сожаление и признание проблем клиента в сообщении 5.,"I'm really sorry to hear that. At ModaMart, we offer a detailed sizing guide to help with the fit, and our return process is designed to be hassle-free.",Подход корректный
4,modamart__4_transcript,Эмпатия,Получилось,Нет,Эмпатия выражена,Продавец выразил понимание и соболезнование по поводу неудачного опыта клиента с другим брендом.,Продавец выразил понимание и признание проблемы клиента в ответе о неудачном опыте.,I understand your concern. The Model X jacket is one of our premium products made with high-quality insulation and weather-resistant materials.,Подход корректный
5,modamart__0_transcript,Приветствие,Получилось,Нет,Приветствие в нужном окне,Приветствие присутствует в первых трёх сообщениях продавца.,Приветствие правильно дано в первом сообщении продавца.,"Hi there! Thank you for taking the time to speak with me today. My name is Jamie, and I’m a sales representative from ModaMart. How are you today?",Подход корректный
6,modamart__1_transcript,Приветствие,Получилось,Нет,Приветствие в нужном окне,Приветствие произошло в первом сообщении продавца.,"Приветствие было в первом сообщении продавца, что соответствует требованию приветствия в первых трех сообщениях.","Good morning, this is Sarah from ModaMart. How can I assist you today?",Подход корректный
7,modamart__2_transcript,Приветствие,Получилось,Нет,Приветствие в нужном окне,Продавец поприветствовал клиента в первых трех сообщениях.,"Приветствие есть в первом сообщении продавца, что соответствует правилам.","Hi there, this is Jake from ModaMart. How are you today?",Подход корректный
8,modamart__3_transcript,Приветствие,Получилось,Нет,Приветствие в нужном окне,Приветствие присутствует в первом сообщении продавца.,Приветствие присутствует в первом сообщении продавца.,Good afternoon! Thank you for calling ModaMart. My name is Alex. How can I assist you today?,Подход корректный
9,modamart__4_transcript,Приветствие,Получилось,Нет,Приветствие в нужном окне,В приветствии продавец представился и поприветствовал клиента в первых трех сообщениях.,"Приветствие присутствует в первом сообщении продавца, в пределах первых трёх сообщений.","Good afternoon, thank you for calling ModaMart. My name is Sarah, how can I assist you today?",Подход корректный


Неуспешных кейсов в выбранном run нет.
Кейсов с QA-расхождением в выбранном run нет.


## 4) Итог для бизнес-пайплайна

Короткий итог по стабильности качества и зонам внимания.


In [6]:
if cases_export.empty:
    print("Нет кейсов для выбранного run.")
else:
    achieved_total = int((cases_export["Результат"] == "Получилось").sum())
    missed_total = int((cases_export["Результат"] == "Не получилось").sum())
    qa_mismatch_total = int((cases_export["Нужна проверка QA"] == "Да").sum())

    if rule_metrics.empty:
        weakest_rule_text = "Нет данных"
    else:
        weakest = rule_metrics.sort_values(["judge_correctness", "evaluator_hit_rate", "rule_key"]).iloc[0]
        weakest_rule_name = RULE_LABELS.get(str(weakest["rule_key"]), "Неизвестное правило")
        weakest_rule_text = (
            f"{weakest_rule_name} "
            f"(согласие QA={float(weakest['judge_correctness']) * 100:.1f}%, "
            f"срабатывание={float(weakest['evaluator_hit_rate']) * 100:.1f}%)"
        )

    print("1) Контрактная модель:", SUMMARY.get("metrics_version", "n/a"))
    print("2) Покрытие QA:", f"{float(SUMMARY.get('judge_coverage', 0.0)):.1%}")
    print("3) Получилось:", achieved_total)
    print("4) Не получилось:", missed_total)
    print("5) Требуют проверки QA:", qa_mismatch_total)
    print("6) Окно приветствия:", globals().get("greeting_window_kpi", "n/a"))
    print("7) Наиболее уязвимое правило:", weakest_rule_text)


1) Контрактная модель: v5_dialog_level_bundle
2) Покрытие QA: 100.0%
3) Получилось: 15
4) Не получилось: 0
5) Требуют проверки QA: 0
6) Окно приветствия: OK (0 нарушений)
7) Наиболее уязвимое правило: Эмпатия (согласие QA=100.0%, срабатывание=100.0%)


## 5) Экспорт всех кейсов в CSV

Экспортируется полный набор кейсов выбранного запуска с теми же заголовками, что в разделе кейсов.


In [7]:
export_path = Path("/Users/ablackman/go/src/github.com/tetraminz/sales_protocol/artifacts/sgr_cases_latest_run.csv")
export_path.parent.mkdir(parents=True, exist_ok=True)

if "CASE_COLUMNS" not in globals():
    CASE_COLUMNS = [
        "Диалог",
        "Правило",
        "Результат",
        "Нужна проверка QA",
        "Причина",
        "Пояснение",
        "Комментарий QA",
        "Цитата",
        "Рекомендация",
    ]

if "cases_export" not in globals():
    cases_export = pd.DataFrame(columns=CASE_COLUMNS)
else:
    cases_export = cases_export.reindex(columns=CASE_COLUMNS)

cases_export.to_csv(export_path, index=False, encoding="utf-8-sig")

print(f"CSV сохранен: {export_path}")
print(f"run_id: {RUN_ID}")
print(f"строк выгружено: {len(cases_export)}")


CSV сохранен: /Users/ablackman/go/src/github.com/tetraminz/sales_protocol/artifacts/sgr_cases_latest_run.csv
run_id: scan_b400f5cf1e1e
строк выгружено: 15
